In [1]:
from common.data_setup import create_dataloaders_mri_2d

In [3]:
import pandas as pd

df = pd.read_csv(
    "results/test_test_fe_resnet50_s_ADNI1-GE_t_ADNI1-Philips_alg_source-only.csv",
    sep=";",
)

In [4]:
import torch
import torch.nn.functional as F

# Exemplo: Modelo de classificação com 3 classes
logits = torch.tensor(
    [[-1.0, 2.0, 0.5], [3.0, 1.0, 0.2]]
)  # Logits de 2 amostras, 3 classes cada

# Aplica softmax para obter probabilidades
probabilities = F.softmax(logits, dim=-1)

print("Probabilities:\n", probabilities)
print(
    "Predicted Class:\n", probabilities.argmax(dim=-1)
)  # Classe com maior probabilidade

Probabilities:
 tensor([[0.0391, 0.7856, 0.1753],
        [0.8360, 0.1131, 0.0508]])
Predicted Class:
 tensor([1, 0])


In [6]:
# Logits para classificação binária
logits = torch.tensor([0.8, -1.5, 2.4])  # Logits para 3 amostras

# Aplica sigmoid
probabilities = torch.sigmoid(logits)

# Predições binárias usando threshold
threshold = 0.5
predictions = (probabilities > threshold).int()
predictions2 = torch.argmax(torch.sigmoid(probabilities), dim=1)
print("Logits:", logits)
print("Sigmoid Probabilities:", probabilities)
print("Binary Predictions:", predictions)
print("Binary Predictions:", predictions2)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [6]:
df

,Exp_Number,Source_Data,Target_Data,Algorithm,LR,Best_Epoch,Best_Val_Source,Test_Source_Acc,Test_Target_Acc,Time_Train
0,uaxj6dvw__lr_0.005940461469115897,ADNI1-GE,ADNI1-Philips,source-only,0.005940,34,0.776401,0.766085,0.733073,67m 20s
1,7qp1jttn__lr_0.0019324041271659325,ADNI1-GE,ADNI1-Philips,source-only,0.001932,32,0.793103,0.803309,0.700521,65m 46s
2,act5cgp1__lr_0.0019372619983492303,ADNI1-GE,ADNI1-Philips,source-only,0.001937,11,0.796875,0.786765,0.721354,64m 5s
3,lpmerdbj__lr_0.00962295262364874,ADNI1-GE,ADNI1-Philips,source-only,0.009623,35,0.764009,0.760570,0.742188,73m 9s
4,g6j17bh7__lr_0.007236386909923879,ADNI1-GE,ADNI1-Philips,source-only,0.007236,31,0.771013,0.750460,0.734375,64m 29s
5,5nyu46ip__lr_0.004971261226661363,ADNI1-GE,ADNI1-Philips,source-only,0.004971,43,0.786099,0.767004,0.744792,63m 48s
6,t17i0tk2__lr_0.008338805282524741,ADNI1-GE,ADNI1-Philips,source-only,0.008339,34,0.768319,0.756893,0.739583,63m 48s
7,dv7nu61o__lr_0.004663171300167574,ADNI1-GE,ADNI1-Philips,source-only,0.004663,22,0.780172,0.764706,0.712240,63m 50s
8,ew7o0hhx__lr_0.008548198667426812,ADNI1-GE,ADNI1-Philips,source-only,0.008548,14,0.759160,0.760570,0.742188,63m 49s
9,mnxkug6c__lr_0.0006245486981609114,ADNI1-GE,ADNI1-Philips,source-only,0.000625,46,0.789871,0.822151,0.700521,63m 47s


In [8]:
df.iloc[df.Test_Source_Acc.idxmax()].Exp_Number

'mnxkug6c__lr_0.0006245486981609114'

In [2]:
import shutil

import pandas as pd


def get_best_model(
    algorithm: str,
    validator: str,
    project_name: str,
    model_name: str,
    metric_select: str,
    max: bool,
):
    results = pd.read_csv(f"results/{project_name}.csv", sep=";")
    if max:
        idx = results[metric_select].idxmax()
    else:
        idx = results[metric_select].idxmin()
    best_model = results.iloc[idx]
    exp_number = best_model.Exp_Number
    print(f"{best_model.Validator} from Val: {best_model.Best_Val_Source}")
    shutil.copyfile(
        f"checkpoints/{algorithm}/{validator}/{exp_number}/{model_name}",
        f"models/{model_name}",
    )


get_best_model(
    algorithm="source-only",
    validator="Accuracy",
    project_name="test_validator_fe_resnet50_s_ADNI1-GE_t_ADNI1-Philips_alg_source-only_v_Accuracy",
    model_name="fe_resnet50_s_ADNI1-GE_t_ADNI1-Philips_alg_source-only_v_Accuracy.pt",
    metric_select="Best_Val_Source",
    max=True,
)

Accuracy from Val: 0.7128232717514038


In [1]:
for domain_side, domain in zip(["src", "target"], ["ADNI1-GE", "ADNI1-Philips"]):
    print(f"{domain_side} - {domain}")

src - ADNI1-GE
target - ADNI1-Philips


In [13]:
import shutil

shutil.copyfile(
    f"checkpoints/source-only/{df.iloc[df.Test_Source_Acc.idxmax()].Exp_Number}/fe_resnet50_s_ADNI1-GE_t_ADNI1-Philips_alg_source-only.pt",
    f"models/fe_resnet50_s_ADNI1-GE_t_ADNI1-Philips_alg_source-only.pt",
)

'models/fe_resnet50_s_ADNI1-GE_t_ADNI1-Philips_alg_source-only.pt'

In [7]:
for domain_side, domain in zip(["src", "target"], ["ADNI1-GE", "ADNI1-Philips"]):
    print(f"{domain_side}-{domain}")

src-ADNI1-GE
target-ADNI1-Philips


In [3]:
dataloaders, target_dataset_size, train_names = create_dataloaders_mri_2d(
    source="ADNI1-GE",
    target="ADNI1-Philips",
    algorithm="source-only",
)

2025-01-08 20:13:53.355 | INFO     | common.data_setup:create_dataloaders_mri_2d:72 - ++++++++++++++++++Loading Source and Target Datasets++++++++++++++++++
2025-01-08 20:13:53.546 | INFO     | common.data_setup:create_dataloaders_mri_2d:94 - ++++++++++++++++++Create Source and Target Datasets+++++++++++++++++++
2025-01-08 20:13:53.547 | INFO     | common.data_setup:create_dataloaders_mri_2d:153 - ++++++++++++++++++++++++++Create Dataloader+++++++++++++++++++++++++++
2025-01-08 20:13:53.547 | INFO     | common.data_setup:create_dataloaders_mri_2d:163 - ++++++++++++++++Finishing the Creation of Dataloaders+++++++++++++++++


In [8]:
dataloaders

{'src_train': <torch.utils.data.dataloader.DataLoader at 0x1ec3dd0fdd0>,
 'src_val': <torch.utils.data.dataloader.DataLoader at 0x1ec3cd20bd0>,
 'src_test': <torch.utils.data.dataloader.DataLoader at 0x1ec3d0896d0>,
 'target_train': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf78a90>,
 'target_val': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf789d0>,
 'target_test': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf78910>,
 'target_train_with_labels': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf78690>,
 'target_val_with_labels': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf78510>,
 'target_test_with_labels': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf78b90>,
 'train': <torch.utils.data.dataloader.DataLoader at 0x1ec3cf78c90>}

In [1]:
from pytorch_adapt.validators import BNMValidator

In [2]:
VALIDATORS = {
    "src_train_target_train_coral_score": lambda kwargs: (
        BNMValidator(),
        {"src_train": "src_train", "target_train": "target_train"},
    ),
    "src_train_target_val_coral_score": lambda kwargs: (
        BNMValidator(),
        {"src_train": "src_train", "target_train": "target_val"},
    ),
    "src_val_target_train_coral_score": lambda kwargs: (
        BNMValidator(),
        {"src_train": "src_val", "target_train": "target_train"},
    ),
    "src_val_target_val_coral_score": lambda kwargs: (
        BNMValidator(),
        {"src_train": "src_val", "target_train": "target_val"},
    ),
}

In [3]:
validators2 = [
    "src_train_target_train_coral_score",
    "src_train_target_val_coral_score",
    "src_val_target_train_coral_score",
    "src_val_target_val_coral_score",
]

In [2]:
metric_hpo = "Val_Source"
if "source" in metric_hpo.lower():
    print("True")
else:
    print("False")

True


In [9]:
from pytorch_adapt.validators import AccuracyValidator

validator = {"Name": "Accuracy", "Class": AccuracyValidator(), "Params": ["src_val"]}

In [13]:
scores = []
for data_score in ["src_data_score", "target_data_score"]:
    dict_scores = {
        key: data_score for key, data_score in zip(validator["Params"], [data_score])
    }
    scores.append(dict_scores)
scores

[{'src_val': 'src_data_score'}, {'src_val': 'target_data_score'}]

In [4]:
chosen_validators = {key: VALIDATORS[key]({"num_classes": 2}) for key in validators2}

In [5]:
chosen_validators

{'src_train_target_train_coral_score': (BNMValidator(required_data=['target_train']),
  {'src_train': 'src_train', 'target_train': 'target_train'}),
 'src_train_target_val_coral_score': (BNMValidator(required_data=['target_train']),
  {'src_train': 'src_train', 'target_train': 'target_val'}),
 'src_val_target_train_coral_score': (BNMValidator(required_data=['target_train']),
  {'src_train': 'src_val', 'target_train': 'target_train'}),
 'src_val_target_val_coral_score': (BNMValidator(required_data=['target_train']),
  {'src_train': 'src_val', 'target_train': 'target_val'})}

In [8]:
for name, (validator, input_keys) in chosen_validators.items():
    print(f"Name:{name} - validator={validator}/input_keys={input_keys}")

Name:src_train_bnm_score - validator=BNMValidator(required_data=['target_train'])/input_keys={'target_train': 'src_train'}
Name:src_val_bnm_score - validator=BNMValidator(required_data=['target_train'])/input_keys={'target_train': 'src_val'}
Name:target_train_bnm_score - validator=BNMValidator(required_data=['target_train'])/input_keys={'target_train': 'target_train'}
Name:target_val_bnm_score - validator=BNMValidator(required_data=['target_train'])/input_keys={'target_train': 'target_val'}


In [9]:
splits = list(
    set([val for v in chosen_validators for val in chosen_validators[v][1].values()])
)
split_splits = [key.split("_") for key in splits]
splits_with_labels = []
for split in split_splits:
    s = f"{split[1]}_with_labels" if split[0] == "target" else split[1]
    splits_with_labels.append([split[0], s])
splits, splits_with_labels

(['src_val', 'src_train', 'target_train', 'target_val'],
 [['src', 'val'],
  ['src', 'train'],
  ['target', 'train_with_labels'],
  ['target', 'val_with_labels']])

In [3]:
[[1, 0.5], [2, 0.75]][-1]

[2, 0.75]

In [10]:
data = {
    a: {"labels": [0], "fl0": [0], "fl3": [0], "fl6": [0], "logits": [0], "preds": [0]}
    for a, b in zip(splits, splits_with_labels)
}
data

{'src_val': {'labels': [0],
  'fl0': [0],
  'fl3': [0],
  'fl6': [0],
  'logits': [0],
  'preds': [0]},
 'src_train': {'labels': [0],
  'fl0': [0],
  'fl3': [0],
  'fl6': [0],
  'logits': [0],
  'preds': [0]},
 'target_train': {'labels': [0],
  'fl0': [0],
  'fl3': [0],
  'fl6': [0],
  'logits': [0],
  'preds': [0]},
 'target_val': {'labels': [0],
  'fl0': [0],
  'fl3': [0],
  'fl6': [0],
  'logits': [0],
  'preds': [0]}}

In [1]:
print(
    **{
        "src_val": {
            "labels": [0],
            "fl0": [0],
            "fl3": [0],
            "fl6": [0],
            "logits": [0],
            "preds": [0],
        }
    }
)

TypeError: 'src_val' is an invalid keyword argument for print()

In [11]:
{key: data[value] for key, value in input_keys.items()}

{'target_train': {'labels': [0],
  'fl0': [0],
  'fl3': [0],
  'fl6': [0],
  'logits': [0],
  'preds': [0]}}

In [12]:
def necessary_splits():
    splits = list(
        set(
            [val for v in chosen_validators for val in chosen_validators[v][1].values()]
        )
    )
    split_splits = [key.split("_") for key in splits]
    splits_with_labels = []
    for split in split_splits:
        s = f"{split[1]}_with_labels" if split[0] == "target" else split[1]
        splits_with_labels.append([split[0], s])
    return splits, splits_with_labels

In [13]:
necessary_splits()

(['src_val', 'src_train', 'target_train', 'target_val'],
 [['src', 'val'],
  ['src', 'train'],
  ['target', 'train_with_labels'],
  ['target', 'val_with_labels']])

In [3]:
import wandb
import yaml
from common.utils import create_directory

args = {"algorithm": "source-only"}
create_directory(type_path="models", args=args)
api = wandb.Api()
entity = "doxdank"
project = "test_fineAll_fe_resnet50_d_ADNI1_s_GE_t_Philips_tl_source-only"
sweep_id = "nvmtd04j"
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

2025-01-09 00:18:02.088 | INFO     | common.utils:create_directory:273 - Directory created successfully! - c:\Users\doxda\OneDrive\Documentos\Repos\masters-degree-data-shift\models\source-only


In [4]:
runs = sorted(
    sweep.runs, key=lambda run: run.summary.get("Val Source Acc", 0), reverse=True
)
best_run = runs[0]
for artifact in best_run.logged_artifacts():
    artifact_path = artifact.download("models/source-only")

wandb: Downloading large artifact model:v1, 217.20MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.3


In [ ]:
import pandas